# Notebook Setup

In [16]:
# Imports

import pandas as pd
import numpy as np
import sklearn
import os

import plotly.express as px
import plotly.graph_objects as go

import requests

# Import Data

In [17]:
# Data imports

# 2014
months_14 = ['apr', 'may', 'jun', 'jul', 'aug', 'sep']
data_14 = pd.DataFrame()

for m in months_14:
    data_14_raw_filepath = r'data\uber-raw-data-{}14.csv'.format(m)
    data_14 = pd.concat([data_14, pd.read_csv(data_14_raw_filepath)])

data_14 = data_14.reset_index()

# 2015
data_15_raw = pd.read_csv(r'data\uber-raw-data-janjune-15.csv\uber-raw-data-janjune-15.csv')
data_15_loc = pd.read_csv(r'data\taxi-zone-lookup.csv')

# Clean and merge data

In [18]:
# Get lat/lon data for data_15_loc

# Initi-locate lat/lon columns
data_15_loc['lat'] = 0
data_15_loc['lon'] = 0

# Replace special characters with space
import re
data_15_loc['Borough'] = data_15_loc['Borough'].str.replace('[^a-zA-Z0-9 ]', '', regex=True)
data_15_loc['Zone'] = data_15_loc['Zone'].str.replace('[^a-zA-Z0-9 ]', ' ', regex=True)

# Some zones are not properly parsed by the nominatim API, so I will rewrite them
# This took a while but allows us a lot more precision on our 2015 data
replacements = {'TriBeCa Civic Center' : 'Tribeca',
                'Greenwich Village North' : 'Greenwich Village',
                'Upper East Side South' : 'Upper East Side',
                'Meatpacking West Village West' : 'West Village',
                'Times Sq Theatre District' : 'Theater District',
                'Central Harlem North' : 'Central Harlem',
                'Upper West Side South' : 'Upper West Side',
                'Penn Station Madison Sq West' : 'Penn Station',
                'Allerton Pelham Gardens' : 'Allerton',
                'Bay Terrace Fort Totten' : 'Fort Totten',
                'Bensonhurst East' : 'Bensonhurst',
                'Bensonhurst West' : 'Bensonhurst',
                'Westchester Village Unionport' : 'Unionport',
                'Williamsbridge Olinville' : 'Williamsbridge',
                'Williamsburg  North Side ' : 'Williamsburg',
                'Williamsburg  South Side ' : 'Williamsburg',
                'Yorkville East' : 'Yorkville',
                'Yorkville West' : 'Yorkville',
                'Bloomfield Emerson Hill' : 'Bloomfield',
                'Breezy Point Fort Tilden Riis Beach' : 'Fort Tilden',
                'Briarwood Jamaica Hills' : 'Briarwood',
                'Bushwick North' : 'Bushwick',
                'Central Harlem' : 'Harlem',
                'Central Harlem' : 'Harlem',
                'Charleston Tottenville' : 'Tottenville',
                'Claremont Bathgate' : 'Claremont',
                'Crown Heights North' : 'Crown Heights',
                'Crown Heights South' : 'Crown Heights',
                'East Flatbush Farragut' : 'Flatbush',
                'East Flatbush Remsen Village' : 'Flatbush',
                'East Harlem North' : 'Harlem',
                'East Harlem South' : 'Harlem',
                'Elmhurst Maspeth' : 'Maspeth',
                'Eltingville Annadale Prince s Bay' : 'Eltingville',
                'Financial District North' : 'Financial District',
                'Flatbush Ditmas Park' : 'Flatbush',
                'Flushing Meadows Corona Park' : 'Flushing Meadows',
                'Fordham South' : 'Fordham',
                'Forest Park Highland Park' : 'Forest Park Highlands',
                'Freshkills Park' : 'Freshkills Park',
                'Governor s Island Ellis Island Liberty Island' : 'Ellis Island',
                'Governor s Island Ellis Island Liberty Island' : 'Ellis Island',
                'Governor s Island Ellis Island Liberty Island' : 'Ellis Island',
                'Greenwich Village South' : 'Greenwich Village',
                'Grymes Hill Clifton' : 'Grymes Hill',
                'Heartland Village Todt Hill' : 'Heartland Village',
                'Hillcrest Pomonok' : 'Hillcrest',
                'Lenox Hill East' : 'Lenox Hill',
                'Lenox Hill West' : 'Lenox Hill',
                'Lincoln Square East' : 'Lincoln Square',
                'Lincoln Square West' : 'Lincoln Square',
                'Madison' : 'Avenue R',
                'Marine Park Floyd Bennett Field' : 'Marine Park',
                'Marine Park Mill Basin' : 'Marine Park',
                'Melrose South' : 'Melrose',
                'Midtown Center' : 'Midtown',
                'Murray Hill Queens' : 'Murray Hill',
                'New Dorp Midland Beach' : 'Midland Beach',
                'North Corona' : 'Corona',
                'Ocean Parkway South' : 'Ocean Parkway',
                'Old Astoria' : 'Astoria',
                'Queensbridge Ravenswood' : 'Queensbridge',
                'Riverdale North Riverdale Fieldston' : 'Riverdale',
                'Saint Michaels Cemetery Woodside' : 'Ditmars Blvd',
                'Schuylerville Edgewater Park' : 'Edgewater Park',
                'Soundview Bruckner' : 'Soundview',
                'Soundview Castle Hill' : 'Soundview',
                'South Beach Dongan Hills' : 'South Beach',
                'Springfield Gardens North' : 'Springfield Gardens',
                'Springfield Gardens South' : 'Springfield Gardens',
                'Stuy Town Peter Cooper Village' : 'Stuytown',
                'Sunset Park East' : 'Sunset Park',
                'Sunset Park West' : 'Sunset Park',
                'Sutton Place Turtle Bay North' : 'Turtle Bay',
                'UN Turtle Bay South' : 'Turtle Bay',
                'Upper East Side North' : 'Upper East Side',
                'Upper West Side North' : 'Upper West Side',
                'Van Cortlandt Village' : 'Kingsbridge',
                'Washington Heights North' : 'Washington Heights',
                'Washington Heights South' : 'Washington Heights',
                'Hillcrest' : '73rd Ave',
                'St Michaels Cemetery' : 'Ditmars Blvd',
                'Unknown' : '',
                'Unknown' : ''}

for key in replacements.keys():
    data_15_loc['Zone'] = data_15_loc['Zone'].str.replace(key, replacements[key])

In [19]:
data_15_loc

,LocationID,Borough,Zone,lat,lon
0,1,EWR,Newark Airport,0,0
1,2,Queens,Jamaica Bay,0,0
2,3,Bronx,Allerton,0,0
3,4,Manhattan,Alphabet City,0,0
4,5,Staten Island,Arden Heights,0,0
...,...,...,...,...,...
260,261,Manhattan,World Trade Center,0,0
261,262,Manhattan,Yorkville,0,0
262,263,Manhattan,Yorkville,0,0
263,264,Unknown,,0,0


In [25]:
# Setup for API request
url_coords = 'https://nominatim.openstreetmap.org/search'
headers_coords = {'User-Agent': 'Chrome/123.0.0.0'}

# Progress meter
from IPython.display import clear_output
ratio_loc_len = len(data_15_loc) / 100

# Query nominatim API for lat/lon for each address
for i, row in data_15_loc.iterrows():
    borough = row['Borough']
    zone = row['Zone']
    query = 'USA, New York, ' + borough + ', ' + zone

    # Query exceptions
    if borough == 'EWR':
        query = 'Newark Liberty Airport'       # airport
    if zone == 'Forest Park Highlands':
        query = 'New York, Highland Forest'    # incorrect borough

    # API Queries
    payload_coords = {'q' : query,
                      'format' : 'json'}

    r_coords =  requests.get(url_coords, params=payload_coords, headers=headers_coords)

    # Assign lat/lon values
    if r_coords.status_code == 200 and r_coords.json():
        data_15_loc.loc[i, 'lat'] = round(float(r_coords.json()[0]['lat']), 7)
        data_15_loc.loc[i, 'lon'] = round(float(r_coords.json()[0]['lon']), 7)
    else:
        print(f"No results found for query: {query}")
        data_15_loc.loc[i, 'lat'] = None
        data_15_loc.loc[i, 'lon'] = None

    # Progress meter
    progress = round(i / ratio_loc_len, 1)
    clear_output(wait=True)                      # clear cell output
    print("Progress: {}%".format(progress))

Progress: 99.6%


In [27]:
# Drop missing locations
data_15_loc = data_15_loc.dropna()

# Merge data_15
data_15 = data_15_raw.merge(data_15_loc, left_on='locationID', right_on='LocationID')
data_15 = data_15.drop(['locationID', 'LocationID'], axis = 1).reset_index()

# Convert Date/Time column to datetime
data_14['date'] = pd.to_datetime(data_14['Date/Time'], format = '%m/%d/%Y %H:%M:%S')
data_14 = data_14.drop('Date/Time', axis=1)
data_15['date'] = pd.to_datetime(data_15['Pickup_date'], format = '%Y-%m-%d %H:%M:%S')
data_15 = data_15.drop('Pickup_date', axis=1)

# Merge both tables
data_15 = data_15.drop(['Affiliated_base_num', 'Borough', 'Zone'], axis=1)
data_15 = data_15.rename(columns={'Dispatching_base_num' : 'base'})
data_14 = data_14.rename(columns={'Lat' : 'lat', 'Lon' : 'lon', 'Base' : 'base'})
data = pd.concat([data_14, data_15])

# Remove index column
data = data.drop('index', axis=1)

# Save data to csv
# gzip is fast and happens to compress this data relatively well (4mins, 170mb)
# xz compresses much better but is extremely slow (12 minutes, 70mb)
data_filepath = r'clean_data.csv.xz'
data.to_csv(data_filepath, compression='xz')